In [3]:
library(data.table)
library(lubridate)
library(rpart)
library(partykit)
library(ggplot2)
library(Metrics)
library(TSrepr)
library(TunePareto)
library(caret)
library(forecast)
library(tidyr)
library(rattle)
options(repr.plot.width=10, repr.plot.height=10)

In [2]:
accu=function(actual,forecast){
  n=length(actual)
  error=actual-forecast
  mean=mean(actual)
  sd=sd(actual)
  CV=sd/mean
  FBias=sum(error)/sum(actual)
  MAPE=sum(abs(error/actual+0.0000001))/n
  RMSE=sqrt(sum(error^2)/n)
  MAD=sum(abs(error))/n
  MADP=sum(abs(error))/sum(abs(actual))
  WMAPE=MAD/mean
  l=data.frame(n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE)
  return(l)
}

## Summary

The aim of this assignment is forecasting hourly total solar power plant production values for multiple small solar power plants from the given data. The data consist of hourly production values and 175 features which are temperature, radiation flux, humidty, total cloud cover (4 different layers) information of 25 different locations. Scenaria is forecasting the day after (d+1) with the available yesterday values (d-1). Explainable Boosted Liner Regression (EBLR) is used to forecast production values. Before constructing the models, missing values are handled. Since there is 175 additional features, principal component analysis is applied to reduce dimensionality and prevent multicollinearity. Then, a base model which is simpe linear regression is constructed. Finally, regression model is improved with decision trees which are learning from the errors form the previous models. In conclusion, forecasts are made according to forecasting scenario and compared with a naive forecast approach which is applying last available data for future values.

## Approach

### Handling Missing Values

In [63]:
dt <- fread("C:/Users/kaan9/OneDrive/Masaüstü/production_with_weather_data.csv")

In [64]:
na_rows <- which(rowSums(is.na(dt))>0)
dt[na_rows,]

date,hour,production,DSWRF_surface_38_35,DSWRF_surface_38_35.25,DSWRF_surface_38_35.5,DSWRF_surface_38_35.75,DSWRF_surface_38_36,DSWRF_surface_38.25_35,DSWRF_surface_38.25_35.25,⋯,TMP_2.m.above.ground_38.75_35,TMP_2.m.above.ground_38.75_35.25,TMP_2.m.above.ground_38.75_35.5,TMP_2.m.above.ground_38.75_35.75,TMP_2.m.above.ground_38.75_36,TMP_2.m.above.ground_39_35,TMP_2.m.above.ground_39_35.25,TMP_2.m.above.ground_39_35.5,TMP_2.m.above.ground_39_35.75,TMP_2.m.above.ground_39_36
<date>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021-06-09,19,7.901200,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-11-26,14,189.840300,NA,NA,NA,NA,NA,NA,NA,⋯,284.835,284.405,283.225,282.735,282.425,283.215,283.015,282.375,282.495,282.715
2021-11-26,16,35.422200,272.98,269.88,235.88,183.32,79.18,260.46,251.68,⋯,285.017,284.447,283.517,282.757,282.217,283.637,283.197,282.347,282.457,282.957
2021-12-01,7,0.260118,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-12-13,0,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-12-22,18,0.000000,NA,NA,NA,NA,NA,NA,NA,⋯,269.097,269.567,268.067,263.837,264.207,268.907,269.037,268.077,267.567,267.297
2021-12-25,16,46.846580,NA,NA,NA,NA,NA,NA,NA,⋯,275.997,275.377,275.497,271.187,269.877,274.217,274.307,273.747,273.017,273.247


After reading the data, there are only 7 rows with missing observations. The null values are filled with the values from 1 hour before since there are not many missing values to influence the model.

In [65]:
for(i in 4:length(dt)){
    col <- dt[,..i]
    col[which(is.na(col))] <- col[which(is.na(col))-1]
    dt[,colnames(dt)[i]:=col]
}

In [66]:
dt[na_rows,]

date,hour,production,DSWRF_surface_38_35,DSWRF_surface_38_35.25,DSWRF_surface_38_35.5,DSWRF_surface_38_35.75,DSWRF_surface_38_36,DSWRF_surface_38.25_35,DSWRF_surface_38.25_35.25,⋯,TMP_2.m.above.ground_38.75_35,TMP_2.m.above.ground_38.75_35.25,TMP_2.m.above.ground_38.75_35.5,TMP_2.m.above.ground_38.75_35.75,TMP_2.m.above.ground_38.75_36,TMP_2.m.above.ground_39_35,TMP_2.m.above.ground_39_35.25,TMP_2.m.above.ground_39_35.5,TMP_2.m.above.ground_39_35.75,TMP_2.m.above.ground_39_36
<date>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021-06-09,19,7.901200,550.78,440.38,489.32,577.92,525.26,373.30,358.16,⋯,295.043,294.743,294.043,291.543,290.143,294.543,293.943,291.843,291.843,290.943
2021-11-26,14,189.840300,438.98,444.28,392.40,387.82,404.92,421.96,417.02,⋯,284.835,284.405,283.225,282.735,282.425,283.215,283.015,282.375,282.495,282.715
2021-11-26,16,35.422200,272.98,269.88,235.88,183.32,79.18,260.46,251.68,⋯,285.017,284.447,283.517,282.757,282.217,283.637,283.197,282.347,282.457,282.957
2021-12-01,7,0.260118,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,276.780,276.520,275.380,273.560,273.620,276.910,277.120,275.110,274.430,274.480
2021-12-13,0,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,278.374,277.674,279.054,277.584,275.714,279.184,278.534,278.054,277.234,277.384
2021-12-22,18,0.000000,193.00,194.44,184.04,183.86,178.42,181.98,177.16,⋯,269.097,269.567,268.067,263.837,264.207,268.907,269.037,268.077,267.567,267.297
2021-12-25,16,46.846580,412.68,417.42,412.18,408.10,401.86,395.74,391.78,⋯,275.997,275.377,275.497,271.187,269.877,274.217,274.307,273.747,273.017,273.247


### Dimensionality Reduction

In [67]:
pca <- princomp(dt[,-c("date","production")])
summary(pca)

Importance of components:
                             Comp.1       Comp.2       Comp.3       Comp.4
Standard deviation     1358.7025368 284.45196749 1.387861e+02 1.037083e+02
Proportion of Variance    0.9141613   0.04006747 9.538189e-03 5.325996e-03
Cumulative Proportion     0.9141613   0.95422882 9.637670e-01 9.690930e-01
                             Comp.5       Comp.6       Comp.7       Comp.8
Standard deviation     84.879930021 77.486855696 68.327695655 60.224290364
Proportion of Variance  0.003567665  0.002973241  0.002311892  0.001796046
Cumulative Proportion   0.972660667  0.975633908  0.977945800  0.979741846
                             Comp.9      Comp.10      Comp.11      Comp.12
Standard deviation     52.039742315 49.611984787 47.047138651 45.930574611
Proportion of Variance  0.001341048  0.001218842  0.001096076  0.001044667
Cumulative Proportion   0.981082895  0.982301736  0.983397812  0.984442479
                            Comp.13      Comp.14      Comp.15      Comp.16

The first 21 principal components are used, since they explain the 99% of the variance.

In [68]:
for(i in 1:21){
    dt[,paste("pca",as.character(i),sep=""):=pca$scores[,i]]
}

In [69]:
dt[,w_day:=as.character(wday(date,label=T))]
dt[,mon:=as.character(month(date,label=T))]
dt[,trnd:=1:.N]
dt$production <- log(dt$production+1)
dt$lag_48 <- shift(dt$production,48)
dt <- dt[date>"2019-09-02"]

Trend, day of the week and month features added to the data. Since production is a non-negative variable, natural log transformation is applied to the output variable to prevent linear regreassion to predict negative values. 

In [70]:
train_dt <- dt[date<="2021-10-28",c("date","hour","production",paste0("pca",1:21),"w_day","mon","trnd","lag_48")]
test_dt <- dt[date=="2021-10-30",c("date","hour","production",paste0("pca",1:21),"w_day","mon","trnd","lag_48")]

Since the aim is predicting the other day's values (d+1), data until "2021-10-28" is used as training data. The last day before the forecast period "2021-10-30" is used as the test data.

### EBLR Model

To construct a base learner model, simple linear regression is used with the trend, day and month information as regressors.

In [71]:
lm_model <- lm(production~trnd+w_day+mon+as.factor(hour),train_dt)
summary(lm_model)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour), 
    data = train_dt)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5866 -0.3699  0.0397  0.3903  1.6995 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)        2.909e-01  3.187e-02   9.129  < 2e-16 ***
trnd               5.393e-06  8.992e-07   5.998 2.04e-09 ***
w_dayCum          -1.131e-02  1.726e-02  -0.655  0.51228    
w_dayÇar           6.066e-03  1.722e-02   0.352  0.72468    
w_dayPaz           4.162e-02  1.726e-02   2.412  0.01587 *  
w_dayPer           9.350e-03  1.726e-02   0.542  0.58804    
w_dayPzt           1.090e-02  1.726e-02   0.631  0.52784    
w_daySal           2.809e-02  1.723e-02   1.631  0.10295    
monAra            -1.060e+00  2.378e-02 -44.562  < 2e-16 ***
monEki            -3.998e-01  2.157e-02 -18.534  < 2e-16 ***
monEyl            -1.912e-01  2.164e-02  -8.839  < 2e-16 ***
monHaz             1.706e-01  2.342e-02   7.285 3.34e-13 ***
monKa

In [72]:
test_pred1 <- predict(lm_model,test_dt)
accu(test_dt$production,test_pred1)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,-0.06105456,Inf,0.5319069,0.3046095,0.1433741,0.1433741


The first model has a R-squared value of 0.9285 which is good for a base learner. The base model has WMAPE value of 0.1433 on the test data. To improve the model, a decision tree will be construct on residuals to find the highest residual nodes.

In [73]:
f_count <- 1
residuals <- lm_model$residuals
tree_learner <- rpart(residuals~.-date-production,
                       data=train_dt,
                       control=rpart.control(cp=0,maxdepth=4))
best_node <- as.numeric(rownames(tree_learner$frame[which.max(abs(tree_learner[[1]]$yval)),]))
b <- path.rpart(tree_learner,best_node)
depth <- length(b[[1]])
for(i in 2:depth){
    if(startsWith(b[[1]][i],"mon") | startsWith(b[[1]][i],"w_day")){
        f_count <- f_count+1
        next
    }
    train_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    test_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    f_count <- f_count+1
}


 node number: 31 
   root
   lag_48>=2.238
   pca2< 178.4
   hour>=16.5
   mon=Ağu,Haz,Mar,May,Nis,Tem


The 31th node is has the highest residual value, so its split information added to the data's feature space. The code above is fitting a tree and adding the split information as variables to the data automatically. However, for splits on month or weekday information, feature generation is handled manually as seen below.

In [74]:
train_dt[,fltr4:=as.numeric(mon %in% c("Ağu","Haz","Mar","May","Nis","Tem"))]
test_dt[,fltr4:=as.numeric(mon %in% c("Ağu","Haz","Mar","May","Nis","Tem"))]
dt[,fltr4:=as.numeric(mon %in% c("Ağu","Haz","Mar","May","Nis","Tem"))]

In [75]:
lm_model2 <- lm(production~trnd+w_day+mon+as.factor(hour)+
                fltr1:fltr2:fltr3:fltr4
                ,train_dt)
summary(lm_model2)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour) + 
    fltr1:fltr2:fltr3:fltr4, data = train_dt)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6294 -0.3151  0.0253  0.3473  2.1955 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)              1.992e-01  2.981e-02   6.682 2.43e-11 ***
trnd                     5.789e-06  8.398e-07   6.893 5.62e-12 ***
w_dayCum                -9.062e-03  1.611e-02  -0.562  0.57390    
w_dayÇar                 2.015e-03  1.608e-02   0.125  0.90032    
w_dayPaz                 3.581e-02  1.612e-02   2.222  0.02630 *  
w_dayPer                 8.457e-03  1.612e-02   0.525  0.59982    
w_dayPzt                 2.719e-03  1.612e-02   0.169  0.86605    
w_daySal                 2.473e-02  1.609e-02   1.537  0.12433    
monAra                  -9.251e-01  2.236e-02 -41.381  < 2e-16 ***
monEki                  -2.665e-01  2.031e-02 -13.122  < 2e-16 ***
monEyl                  -5.773e-02 

In [76]:
test_pred2 <- predict(lm_model2,test_dt)
accu(test_dt$production,test_pred2)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,-0.06412853,Inf,0.4303376,0.2784757,0.1310734,0.1310734


After adding these 4 filters to the data, new linear model's R-squared value is increased to 0.9376 and the new model has WMAPE value of 0.1310. To improve more, another decisions tree will be constructed to learn from the residuals.

In [77]:
residuals <- lm_model2$residuals
tree_learner <- rpart(residuals~.-date-production,
                       data=train_dt,
                       control=rpart.control(cp=0,maxdepth=4))
best_node <- as.numeric(rownames(tree_learner$frame[which.max(abs(tree_learner[[1]]$yval)),]))
b <- path.rpart(tree_learner,best_node)
depth <- length(b[[1]])
for(i in 2:depth){
    if(startsWith(b[[1]][i],"mon") | startsWith(b[[1]][i],"w_day")){
        f_count <- f_count+1
        next
    }
    train_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    test_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    f_count <- f_count+1
}


 node number: 16 
   root
   lag_48< 1.229
   lag_48>=0.01612
   mon=Ara,Eki,Kas,Oca,Şub
   hour< 18.5


The node with highest value is 16th node. Split information is added to data as new features.

In [78]:
train_dt[,fltr7:=as.numeric(mon %in% c("Ara","Eki","Kas","Oca","Şub"))]
test_dt[,fltr7:=as.numeric(mon %in% c("Ara","Eki","Kas","Oca","Şub"))]
dt[,fltr7:=as.numeric(mon %in% c("Ara","Eki","Kas","Oca","Şub"))]

In [79]:
lm_model3 <- lm(production~trnd+w_day+mon+as.factor(hour)+
                fltr1:fltr2:fltr3:fltr4+
                fltr5:fltr6:fltr7:fltr8
                ,train_dt)
summary(lm_model3)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour) + 
    fltr1:fltr2:fltr3:fltr4 + fltr5:fltr6:fltr7:fltr8, data = train_dt)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.7102 -0.2674  0.0231  0.3172  2.3411 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)              1.785e-01  2.726e-02   6.549 5.94e-11 ***
trnd                     5.155e-06  7.680e-07   6.712 1.98e-11 ***
w_dayCum                -1.584e-02  1.474e-02  -1.075   0.2825    
w_dayÇar                 1.115e-03  1.471e-02   0.076   0.9396    
w_dayPaz                 3.654e-02  1.474e-02   2.480   0.0132 *  
w_dayPer                 4.443e-03  1.474e-02   0.301   0.7631    
w_dayPzt                -6.432e-03  1.474e-02  -0.436   0.6626    
w_daySal                 2.008e-02  1.471e-02   1.365   0.1723    
monAra                  -7.888e-01  2.057e-02 -38.353  < 2e-16 ***
monEki                  -1.730e-01  1.863e-02  -9.287  < 2e-16 ***
monEyl   

In [80]:
test_pred3 <- predict(lm_model3,test_dt)
accu(test_dt$production,test_pred3)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,-0.06536037,Inf,0.3891039,0.2495588,0.1174627,0.1174627


After the filter variables, model has a R-squared value of 0.9479 and WMAPE value is improved to 0.1174. Since the R-squared and WMAPE are improved, another decision tree could be constructed.

In [81]:
residuals <- lm_model3$residuals
tree_learner <- rpart(residuals~.-date-production,
                       data=train_dt,
                       control=rpart.control(cp=0,maxdepth=4))
best_node <- as.numeric(rownames(tree_learner$frame[which.max(abs(tree_learner[[1]]$yval)),]))
b <- path.rpart(tree_learner,best_node)
depth <- length(b[[1]])
for(i in 2:depth){
    if(startsWith(b[[1]][i],"mon") | startsWith(b[[1]][i],"w_day")){
        f_count <- f_count+1
        next
    }
    train_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    test_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    f_count <- f_count+1
}


 node number: 16 
   root
   pca2>=406
   lag_48>=0.1298
   pca1< 55.64
   mon=Ara,Eyl,Kas,Oca,Şub


The 16th node has the highest residual, thus its split information added to the data as filter variables, again.

In [82]:
train_dt[,fltr12:=as.numeric(mon %in% c("Ara","Eyl","Kas","Oca","Şub"))]
test_dt[,fltr12:=as.numeric(mon %in% c("Ara","Eyl","Kas","Oca","Şub"))]
dt[,fltr12:=as.numeric(mon %in% c("Ara","Eyl","Kas","Oca","Şub"))]

In [83]:
lm_model4 <- lm(production~trnd+w_day+mon+as.factor(hour)+
                fltr1:fltr2:fltr3:fltr4+
                fltr5:fltr6:fltr7:fltr8+
                fltr9:fltr10:fltr11:fltr12
                ,train_dt)
summary(lm_model4)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour) + 
    fltr1:fltr2:fltr3:fltr4 + fltr5:fltr6:fltr7:fltr8 + fltr9:fltr10:fltr11:fltr12, 
    data = train_dt)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.95951 -0.25158  0.01102  0.28737  2.94857 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 1.615e-01  2.540e-02   6.356 2.12e-10 ***
trnd                        4.090e-06  7.158e-07   5.714 1.12e-08 ***
w_dayCum                   -6.703e-03  1.373e-02  -0.488   0.6254    
w_dayÇar                    2.204e-03  1.370e-02   0.161   0.8722    
w_dayPaz                    2.985e-02  1.373e-02   2.174   0.0297 *  
w_dayPer                    9.560e-03  1.373e-02   0.696   0.4864    
w_dayPzt                   -1.991e-02  1.374e-02  -1.449   0.1473    
w_daySal                    2.087e-02  1.371e-02   1.523   0.1278    
monAra                     -6.314e-01  1.939e-02 -32.572  < 2e-16 ***
mo

In [84]:
test_pred4 <- predict(lm_model4,test_dt)
accu(test_dt$production,test_pred4)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,-0.06067473,Inf,0.3715831,0.2257848,0.1062728,0.1062728


In [85]:
residuals <- lm_model4$residuals
tree_learner <- rpart(residuals~.-date-production,
                       data=train_dt,
                       control=rpart.control(cp=0,maxdepth=4))
best_node <- as.numeric(rownames(tree_learner$frame[which.max(abs(tree_learner[[1]]$yval)),]))
b <- path.rpart(tree_learner,best_node)
depth <- length(b[[1]])
for(i in 2:depth){
    if(startsWith(b[[1]][i],"mon") | startsWith(b[[1]][i],"w_day")){
        f_count <- f_count+1
        next
    }
    train_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    test_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    f_count <- f_count+1
}


 node number: 31 
   root
   pca3< 195.4
   lag_48>=0.8804
   pca2< 39.29
   fltr8< 0.5


In [86]:
lm_model5 <- lm(production~trnd+w_day+mon+as.factor(hour)+
                fltr1:fltr2:fltr3:fltr4+
                fltr5:fltr6:fltr7:fltr8+
                fltr9:fltr10:fltr11:fltr12+
                fltr13:fltr14:fltr15:fltr16
                ,train_dt)
summary(lm_model5)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour) + 
    fltr1:fltr2:fltr3:fltr4 + fltr5:fltr6:fltr7:fltr8 + fltr9:fltr10:fltr11:fltr12 + 
    fltr13:fltr14:fltr15:fltr16, data = train_dt)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.9709 -0.2405  0.0126  0.2783  2.9539 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  1.530e-01  2.510e-02   6.096 1.11e-09 ***
trnd                         4.230e-06  7.073e-07   5.980 2.27e-09 ***
w_dayCum                    -6.600e-03  1.357e-02  -0.486  0.62667    
w_dayÇar                     1.935e-03  1.354e-02   0.143  0.88637    
w_dayPaz                     2.876e-02  1.357e-02   2.120  0.03400 *  
w_dayPer                     8.714e-03  1.357e-02   0.642  0.52081    
w_dayPzt                    -2.068e-02  1.357e-02  -1.523  0.12767    
w_daySal                     2.019e-02  1.354e-02   1.491  0.13608    
monAra                      -6.145e-01  1.9

In [87]:
test_pred5 <- predict(lm_model5,test_dt)
accu(test_dt$production,test_pred5)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,-0.06154527,Inf,0.3664161,0.2218386,0.1044153,0.1044153


In [88]:
residuals <- lm_model5$residuals
tree_learner <- rpart(residuals~.-date-production-lag_48,
                       data=train_dt,
                       control=rpart.control(cp=0,maxdepth=4))
best_node <- as.numeric(rownames(tree_learner$frame[which.max(abs(tree_learner[[1]]$yval)),]))
b <- path.rpart(tree_learner,best_node)
depth <- length(b[[1]])
for(i in 2:depth){
    if(startsWith(b[[1]][i],"mon") | startsWith(b[[1]][i],"w_day")){
        f_count <- f_count+1
        next
    }
    train_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    test_dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    dt[,paste("fltr",as.character(f_count),sep=""):=as.numeric(eval(parse(text = b[[1]][i])))]
    f_count <- f_count+1
}


 node number: 16 
   root
   pca3>=195.4
   fltr10>=0.5
   fltr3< 0.5
   pca1< 616.7


In [89]:
lm_model6 <- lm(production~trnd+w_day+mon+as.factor(hour)+
                fltr1:fltr2:fltr3:fltr4+
                fltr5:fltr6:fltr7:fltr8+
                fltr9:fltr10:fltr11:fltr12+
                fltr13:fltr14:fltr15:fltr16+
                fltr17:fltr18:fltr19:fltr20
                ,train_dt)
summary(lm_model6)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour) + 
    fltr1:fltr2:fltr3:fltr4 + fltr5:fltr6:fltr7:fltr8 + fltr9:fltr10:fltr11:fltr12 + 
    fltr13:fltr14:fltr15:fltr16 + fltr17:fltr18:fltr19:fltr20, 
    data = train_dt)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.0690 -0.2328  0.0105  0.2638  2.8242 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  1.411e-01  2.417e-02   5.835 5.45e-09 ***
trnd                         3.699e-06  6.811e-07   5.431 5.68e-08 ***
w_dayCum                    -4.238e-03  1.306e-02  -0.324  0.74560    
w_dayÇar                    -3.665e-03  1.304e-02  -0.281  0.77860    
w_dayPaz                     3.478e-02  1.306e-02   2.662  0.00777 ** 
w_dayPer                     1.058e-02  1.307e-02   0.810  0.41803    
w_dayPzt                    -2.106e-02  1.307e-02  -1.612  0.10708    
w_daySal                     1.933e-02  1.304e-02   1.482  0.13830    
monAra  

In [90]:
test_pred6 <- predict(lm_model6,test_dt)
accu(test_dt$production,test_pred6)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,-0.06186988,Inf,0.3604163,0.2082301,0.09801007,0.09801007


The EBLR method is applied to the model for 5 iterations. While the base regression model has R-squared value of 0.9285 and has WMAPE error rate of 0.1433, the final model has 0.959 R-squared value and WMAPE error rate of 0.0980 on the test data. This means that addition of new features which is from decision trees applied on the residuals, improved the model.

In [91]:
lm_model_son <- lm(production~trnd+w_day+mon+as.factor(hour)+lag_48+
                fltr1:fltr2:fltr3:fltr4+
                fltr5:fltr6:fltr7:fltr8+
                fltr9:fltr10:fltr11:fltr12+
                fltr13:fltr14:fltr15:fltr16+
                fltr17:fltr18:fltr19:fltr20    
                ,train_dt)
summary(lm_model_son)


Call:
lm(formula = production ~ trnd + w_day + mon + as.factor(hour) + 
    lag_48 + fltr1:fltr2:fltr3:fltr4 + fltr5:fltr6:fltr7:fltr8 + 
    fltr9:fltr10:fltr11:fltr12 + fltr13:fltr14:fltr15:fltr16 + 
    fltr17:fltr18:fltr19:fltr20, data = train_dt)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.65593 -0.10680  0.00334  0.12286  2.79766 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  3.395e-02  1.848e-02   1.837  0.06625 .  
trnd                         9.461e-07  5.208e-07   1.817  0.06928 .  
w_dayCum                    -2.895e-03  9.977e-03  -0.290  0.77169    
w_dayÇar                    -3.509e-03  9.957e-03  -0.352  0.72451    
w_dayPaz                     5.017e-02  9.978e-03   5.028 4.99e-07 ***
w_dayPer                     3.577e-03  9.979e-03   0.358  0.72000    
w_dayPzt                    -5.554e-04  9.982e-03  -0.056  0.95563    
w_daySal                     4.793e-03  9.960e-03   0.481  0

In [92]:
test_pred_son <- predict(lm_model_son,test_dt)
accu(test_dt$production,test_pred_son)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2.124579,2.475094,1.164981,0.04236887,Inf,0.2149382,0.152031,0.07155817,0.07155817


To make a final improvement, lagged value of production added to the final model. 48th lag improved the R-squared to 0.9761 and WMAPE error rate on test to 0.0715.

## Forecast and Conclusion

To forecast, training data is updated every day after 10th of october, then a new model constructed with the new training data and forecasts are made for the 2 days after.

In [93]:
train_end <- nrow(dt[date<="2021-10-30"])
test_dates <- unique(dt$date[dt$date<="2021-12-25" & dt$date>="2021-11-01"])
preds <- numeric(0)
for(d in test_dates ){
    train_tmp <- dt[1:train_end,]
    model <- lm(production~trnd+w_day+mon+as.factor(hour)+lag_48+
                fltr1:fltr2:fltr3:fltr4+
                fltr5:fltr6:fltr7:fltr8+
                fltr9:fltr10:fltr11:fltr12+
                fltr13:fltr14:fltr15:fltr16+
                fltr17:fltr18:fltr19:fltr20   
                ,train_tmp)
    p <- predict(model,newdata=dt[date==d,])
    preds <- c(preds,p)
    train_end <- train_end + 24
}

In [97]:
real <- exp(dt[date %in% test_dates,production])-1
base_sc <- exp(dt[date %in% test_dates,lag_48])
accu(real,base_sc)
preds_trns <- exp(preds)-1
accu(real,preds_trns)

n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1296,45.46808,79.29,1.743861,-0.01978126,Inf,51.70043,23.52425,0.5173795,0.5173795


n,mean,sd,CV,FBias,MAPE,RMSE,MAD,MADP,WMAPE
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1296,45.46808,79.29,1.743861,0.1471751,Inf,38.04042,16.41105,0.3609357,0.3609357


Predictions are transformed to real production values by taking their exponentials. The predictions has WMAPE value of 0.3609 over the test period near 2 months. Naive forecast has the WMAPE error rate of 0.5173 which means that EBLR is approximately 30%  better than the base line forecast. Learning from the previous error is significantly improves linear regression model.